# Concatenate garbage data from Arroyo Vivo 
## (Produces final arroyovivo_residuosjornada_point geodataframe) 

This notebook takes as input 9 csvs that come directly from Marine App (mdt csvs), which contain datetime data.

There are two types of mdt csvs. The first type has a column for "master_item_id" (which represents a middle category) as a number, while the second type has a column for "material" (which also represents a middle category) as a string.

This column is changed to the name "tipo", while a "material" column is created for all csvs based on previous data (generated on Notebook 01a_concatenate_garbage_data).

* "itemname" --> How the item was used (Bolsa de Plastico para comestibles, Vasos o tapas de unicel o Plastico)
* "tipo" --> middle category
* "material" --> Material of item (Vidrio, Plastico, Papel, Unicel etc)

Notes:
* After this notebook it was noted that some points where missing. The missing points correspond to csv "30_05.csv" and required a specific data treatment in order to be added.

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download data

### Download new base data (9 csvs with timestamp)

In [2]:
tmp_dir = f'../../data/external/temporal_todocker/arroyo_vivo/input_csv_2/'

29_0ctubre_mdt-data.csv

In [3]:
doc_29_0ctubre_mdt_data = pd.read_csv(tmp_dir+'29_0ctubre_mdt-data.csv',index_col=False)
doc_29_0ctubre_mdt_data.columns

/tmp/ipykernel_1000/2686198930.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_29_0ctubre_mdt_data = pd.read_csv(tmp_dir+'29_0ctubre_mdt-data.csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

mdt-data (1).csv

In [4]:
doc_mdt_data_1 = pd.read_csv(tmp_dir+'mdt-data (1).csv',index_col=False)
doc_mdt_data_1.columns

/tmp/ipykernel_1000/2925154035.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_1 = pd.read_csv(tmp_dir+'mdt-data (1).csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

mdt-data 5julio-24sep.csv

In [5]:
doc_mdt_data_5julio_24sep = pd.read_csv(tmp_dir+'mdt-data 5julio-24sep.csv',index_col=False)
doc_mdt_data_5julio_24sep.columns

/tmp/ipykernel_1000/1728166398.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_5julio_24sep = pd.read_csv(tmp_dir+'mdt-data 5julio-24sep.csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

mdt-data 22ago-24sep.csv

In [6]:
doc_mdt_data_22ago_24sep = pd.read_csv(tmp_dir+'mdt-data 22ago-24sep.csv',index_col=False)
doc_mdt_data_22ago_24sep.columns

/tmp/ipykernel_1000/1597682300.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_22ago_24sep = pd.read_csv(tmp_dir+'mdt-data 22ago-24sep.csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

mdt-data abril a junio 2023.csv

In [7]:
doc_mdt_data_abril_junio_2023 = pd.read_csv(tmp_dir+'mdt-data abril a junio 2023.csv',index_col=False)
doc_mdt_data_abril_junio_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

mdt-data agosto 2023.csv

In [8]:
doc_mdt_data_agosto_2023 = pd.read_csv(tmp_dir+'mdt-data agosto 2023.csv',index_col=False)
doc_mdt_data_agosto_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

mdt-data_27agosto.csv

In [9]:
doc_mdt_data_27agosto = pd.read_csv(tmp_dir+'mdt-data_27agosto.csv',index_col=False)
doc_mdt_data_27agosto.columns

/tmp/ipykernel_1000/3933488440.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_27agosto = pd.read_csv(tmp_dir+'mdt-data_27agosto.csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

mdt-data_octubre a marzo 2023.csv

In [10]:
doc_mdt_data_octubre_marzo_2023 = pd.read_csv(tmp_dir+'mdt-data_octubre a marzo 2023.csv',index_col=False)
doc_mdt_data_octubre_marzo_2023.columns

Index(['list_name', 'itemname', 'material', 'quantity', 'description',
       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
       'log_index'],
      dtype='object')

mdt-data_Todos los tramos_30nov.csv

In [11]:
doc_mdt_data_todos_los_tramos_30nov = pd.read_csv(tmp_dir+'mdt-data_Todos los tramos_30nov.csv',index_col=False)
doc_mdt_data_todos_los_tramos_30nov.columns

/tmp/ipykernel_1000/1232169902.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  doc_mdt_data_todos_los_tramos_30nov = pd.read_csv(tmp_dir+'mdt-data_Todos los tramos_30nov.csv',index_col=False)


Index(['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
       'latitude', 'longitude', 'altitude', 'description', 'radius',
       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location'],
      dtype='object')

## Data treatment for new base data

### Data treatment - Concatenate new base data

CSVs type 1 - Starting with master_item_id column

In [12]:
# csvs type 1 contain 'master_item_id' as a starting column.
# All columns: ['master_item_id', 'item_id', 'itemname', 'id', 'log_index', 'quantity',
#       'latitude', 'longitude', 'altitude', 'description', 'radius',
#       'manual_upload', 'project_id', 'project_name', 'timestamp', 'location']

csvs_type1 = ['29_0ctubre_mdt-data.csv','mdt-data (1).csv','mdt-data 5julio-24sep.csv','mdt-data 22ago-24sep.csv','mdt-data_27agosto.csv','mdt-data_Todos los tramos_30nov.csv']

# Concatenate csvs
type_1 = pd.DataFrame()
for csv in csvs_type1:
    tmp_gdf = pd.read_csv(tmp_dir+csv,
                          index_col=False)
    type_1 = pd.concat([type_1,tmp_gdf])
    
# Set column with date time format
type_1['datetime'] = pd.to_datetime(type_1['timestamp'], format='%Y%m%d%H%M%S')

# Keep columns of interest
type_1 = type_1[['id','itemname','master_item_id','quantity','latitude','longitude','altitude','datetime']]
# Rename column of interest
type_1.rename(columns={'master_item_id':'tipo'},inplace=True)

# Show
print(type_1.shape)
type_1.head(1)

/tmp/ipykernel_1000/3208410158.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_1000/3208410158.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_1000/3208410158.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_1000/3208410158.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipykernel_1000/3208410158.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  tmp_gdf = pd.read_csv(tmp_dir+csv,
/tmp/ipyke

(15043, 8)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,1,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


CSVs type 2 - Starting with list_name column

In [13]:
# csvs type 2 contain 'list_name' as a starting column.
# All columns: ['list_name', 'itemname', 'material', 'quantity', 'description',
#       'latitude', 'longitude', 'altitude', 'radius', 'location', 'timestamp',
#       'dt', 'project_name', 'username', 'manual_upload', 'event_name', 'id',
#       'log_index']

csvs_type2 = ['mdt-data abril a junio 2023.csv','mdt-data agosto 2023.csv','mdt-data_octubre a marzo 2023.csv']

# Concatenate csvs
type_2 = pd.DataFrame()
for csv in csvs_type2:
    tmp_gdf = pd.read_csv(tmp_dir+csv,
                          index_col=False)
    type_2 = pd.concat([type_2,tmp_gdf])
    
# Set column with date time format
type_2['datetime'] = pd.to_datetime(type_2['timestamp'], format='%Y%m%d%H%M%S')

# Keep columns of interest
type_2 = type_2[['id','itemname','material','quantity','latitude','longitude','altitude','datetime']]
# Rename column of interest
type_2.rename(columns={'material':'tipo'},inplace=True)

# Show
print(type_2.shape)
type_2.head(1)

(9016, 8)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime
0,2398898,Bolsa de plástico para comestibles,Productos Plásticos para Comida,10,25.641958,-100.296648,553.976016,2023-04-15 14:30:48


Join both types

In [14]:
joined_types = pd.concat([type_1,type_2])

# Show
print(joined_types.shape)
joined_types.head(1)

(24059, 8)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,1,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


### Data treatment - Work on 'tipo' data

Problem: Most itemnames have type expressed as integer and type expressed as string.

In [15]:
prueba = joined_types.loc[joined_types.itemname == 'Otro']
prueba.tipo.unique()

array([36, 'Otro'], dtype=object)

Transform 'tipo' column values which are integer to string according to other values within the same gdf

In [16]:
joined_types_c = joined_types.copy()

# Para cada itemname
for item in list(joined_types_c.itemname.unique()):
    
    # Obtener una lista de los tipos
    tmp = joined_types_c.loc[joined_types_c.itemname == item]
    tipos = list(tmp.tipo.unique())
    
    # Si el itemname nada mas tiene un tipo:
    if (len(tipos) == 1):
        # y el tipo es un string, aceptarlo como el tipo para todos esos itemnames.
        if isinstance(tipos[0], str):                  
            idx = ((joined_types_c.tipo == tipos[0]) & (joined_types_c.itemname == item))
            joined_types_c.loc[idx,'tipo'] = tipos[0]
            
        # y el tipo es un int, colocar tipo 'desconocido'
        elif isinstance(tipos[0], int):
            idx = ((joined_types_c.tipo == tipos[0]) & (joined_types_c.itemname == item))
            joined_types_c.loc[idx,'tipo'] = 'desconocido'
            
        # Si es algo más, colocar como caso especial.
        else:
            print(f'Revisar caso para el itemname {itemname}')
            
    # Pero si hay dos tipos, lo más probable es que uno sea int y el otro str.
    elif len(tipos) == 2:
        # Si el primero es el int, colocar el segundo como el tipo
        if isinstance(tipos[0], int):
            idx = ((joined_types_c.tipo == tipos[0]) & (joined_types_c.itemname == item))
            joined_types_c.loc[idx,'tipo'] = tipos[1]
        
        # Si el segundo es el int, colocar el primero como el tipo  
        elif isinstance(tipos[1], int):
            idx = ((joined_types_c.tipo == tipos[1]) & (joined_types_c.itemname == item))
            joined_types_c.loc[idx,'tipo'] = tipos[0] 
                    
    # Si hay más de tres tipos para un itemname, es un caso especial. Atender por separado.
    else:
        print(f'El itemname {itemname} tiene más de 2 tipos.')
        
# Show
print(joined_types_c.shape)
joined_types_c.head(1)

(24059, 8)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


Problem solved

In [17]:
prueba2 = joined_types_c.loc[joined_types_c.itemname == 'Otro']
prueba2.tipo.unique()

array(['Otro'], dtype=object)

In [18]:
dicc_check = {}

for item in list(joined_types_c.itemname.unique()):
    tmp = joined_types_c.loc[joined_types_c.itemname == item]
    tipos = list(tmp.tipo.unique())
    
    dicc_check[item] = tipos

dicc_check

{'Bolsa de plástico para comestibles': ['Productos Plásticos para Comida'],
 'Bolsa de plástico (otra)': ['Productos Plásticos para Comida'],
 'Botella de plástico': ['Productos Plásticos para Comida'],
 'Tapa de botella de plástico': ['Productos Plásticos para Comida'],
 'Cajetilla de cigarrillos': ['Productos de Tabaco'],
 'Otro producto de tabaco': ['Productos de Tabaco'],
 'Cigarrillos': ['Productos de Tabaco'],
 'Vasos o tapas de unicel o plástico': ['Productos Plásticos para Comida'],
 'Envoltorio de plástico para alimentos': ['Productos Plásticos para Comida'],
 'Fragmentos de plástico duro': ['Fragmentos de Plástico  '],
 'Fragmentos de película de plástico': ['Fragmentos de Plástico  '],
 'Utensilios de plástico': ['Productos Plásticos para Comida'],
 'Latas de aluminio o estaño': ['Metales'],
 'Botella de vidrio': ['Vidrio'],
 'Llantas': ['Otros Productos Plásticos'],
 'Ropa': ['Tela'],
 'Piezas de tela': ['Tela'],
 'Tablas de madera': ['Materiales de Construcción'],
 'Materi

Manually set 'tipo' for 'desconocido' category

In [19]:
#desconocidos:'Elemento de prueba','Bote de reciclaje','Cotonetes de algodón','Otro equipo de pesca','Sobres de tabaco','Tienda de café / té'

joined_types_cc = joined_types_c.copy()

desconocidos_dicc = {'Elemento de prueba':'Salud e higiene',
                     'Bote de reciclaje':'Infraestructura para Desechos ',
                     'Cotonetes de algodón':'Salud e higiene',
                     'Otro equipo de pesca':'Herramientas de Pesca',
                     'Sobres de tabaco':'Productos de Tabaco',
                     'Tienda de café / té':'Tienda'}

for desconocido in desconocidos_dicc:
    idx = joined_types_cc.itemname == desconocido
    joined_types_cc.loc[idx,'tipo'] = desconocidos_dicc[desconocido]

# Show
print(joined_types_cc.shape)
joined_types_cc.head(1)

(24059, 8)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


In [20]:
joined_types_cc.tipo.unique()

array(['Productos Plásticos para Comida', 'Productos de Tabaco',
       'Fragmentos de Plástico  ', 'Metales', 'Vidrio',
       'Otros Productos Plásticos', 'Tela', 'Materiales de Construcción',
       'Papel', 'Otro', 'Desechos Electrónicos', 'Desechos Orgánicos',
       'Tienda', 'EPI', 'Productos de Cuidado Personal',
       'Herramientas de Pesca', 'Salud e higiene',
       'Infraestructura para Desechos ', 'Restaurantes', 'Hidrología'],
      dtype=object)

### Data treatment - Work on 'material' data (Dictionary)

#### Dictionary - Itemname-Material Problem solving

* Problem: On the first concatenation notebook it was noted that some itemnames have two or more materials in it.
* Solution: Find those itemnames and solve for each invividually

In [21]:
# Referencia
schema = 'projects_research'
table = 'arroyovivo_residuos_point'
query = f"SELECT * FROM {schema}.{table}"
residuos_point = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(residuos_point.shape)
residuos_point.head(1)

(16179, 9)


,itemname,tipo,material,quantity,latitude,longitude,altitude,data_origin,geometry
0,Latas de aluminio o estaño,None,Aluminio,11,25.644154,-100.310618,627.104902,9 de enero.csv,MULTIPOINT (-100.31062 25.64415)


In [22]:
# Group original data by itemname and material
categories = residuos_point.groupby(['itemname','material']).count().reset_index()[['itemname','material']]

# Count how many materials exist for each itemname
variations = categories.groupby('itemname').count()

# Find where there are 2 or more variations
problem = variations.loc[variations.material>1].reset_index()
problem_list = list(problem.itemname.unique())
categories.loc[categories.itemname.isin(problem_list)]

,itemname,material
12,Cartón acanalado,Carton
13,Cartón acanalado,Papel
14,Cartón recubierto,Carton
15,Cartón recubierto,Papel
18,Chancletas/sandalias,Plastico
19,Chancletas/sandalias,Tela
21,Contenedor de espuma de poliestireno (unicel),Plastico
22,Contenedor de espuma de poliestireno (unicel),Unicel
24,Cubrezapatos de hospital,EPI
25,Cubrezapatos de hospital,Salud e higiene


#### Dictionary - Manual Building of material-itemname dictionary

In [23]:
material_itemname_dicc = {'plastico':['Bolsa de plástico para comestibles', 'Bolsa de plástico (otra)','Botella de plástico','Tapa de botella de plástico',
                                      'Envoltorio de plástico para alimentos','Fragmentos de plástico duro', 'Fragmentos de película de plástico',
                                      'Utensilios de plástico','Otro plástico relacionado con los alimentos','Otro plástico',
                                      'Cuerda de plástico/ cinta o correas de embalaje','Bolsas a granel','Popotes', 
                                      'Plastico para cerveza'
                                     ],
                          'tabaco':['Cajetilla de cigarrillos', 'Otro producto de tabaco','Cigarrillos','Sobres de tabaco'],
                          'metal':['Latas de aluminio o estaño','Tapas o lengüetas de metal para botellas','Fragmentos de metal','Papel de aluminio','Otro metal'],
                          'vidrio':['Botella de vidrio','Otro vidrio'],
                          'llantas':['Llantas'],
                          'tela':['Ropa','Piezas de tela','Otra tela','Toallas o trapos','Ropa o zapatos'],
                          'escombros':['Materiales de construcción','Agregado y ladrillo','Otros artículos de construcción','Pernos/ clavos y tornillos'],
                          'papel':['Envoltura de alimentos de papel sin recubrimiento','Caja de papel multimaterial','Papel','Recibos','Otro papel','Bolsas de papel'],
                          'carton':['Cartón acanalado','Cartón recubierto'],
                          'otros':['Otro','Otros fragmentos'],
                          'salud e higiene':['Otro producto de cuidado personal','Tubo de pasta de dientes u otro producto','Champú u otro recipiente de HDPE',
                                             'Cotonetes de algodón','Cepillos de dientes','Sobre de productos de cuidado personal','Elemento de prueba'],
                          'unicel':['Contenedor de espuma de poliestireno (unicel)','Fragmentos de espuma (unicel)'],
                          'electronica':['Otros desechos electrónicos','Fragmentos de electrónicos'],
                          'organico':['Otros desechos orgánicos','Desechos de comida'],
                          'ceramica':[],
                          'comercio':['Tienda de comestibles','Tienda al por menor','Otra tienda','Tienda de conveniencia','Puesto de comida para llevar','Tienda de café / té',
                                      'Otro Restaurante'],
                          'EPI':['Máscaras faciales', 'Careta','Cubrezapatos de hospital','Otros EPI','Embalaje de EPI asociado','Guantes desechables',
                                'Paquete de medicamentos','Toallitas desinfectantes','Redes para el cabello','Embalaje de mascarilla facial'],
                          'infraestructura':['Bote de la basura', 'Bote de reciclaje','Canal', 'Reciclaje informal', 'Colector de aguas pluviales','Basurero'],
                          'baterias':['Baterías',],
                          'goma':['Ligas de goma'],
                          'madera':['Tablas de madera','Palito de paleta'],
                          'pesca':['Otro equipo de pesca','Sedal','Cuerda de plástico','Red de plástico o piezas de red','Boyas y Flotadores'],
                          'cenizas':['Pila de cenizas'],
                         'unicel/plastico':['Vasos o tapas de unicel o plástico','Tazón de comida para llevar'],
                         'vidrio/ceramica':['Fragmentos de vidrio o cerámica'],
                         'plastico/tela':['Chancletas/sandalias']}

In [24]:
# Revisar que todos los itemnames estén asignados a un material en el diccionario:

# Generar lista
analized_items = []
for k in material_itemname_dicc.keys():
    for item in material_itemname_dicc[k]:
        analized_items.append(item)

# Analizar gdf original
for item in list(residuos_point.itemname.unique()):
    if item not in analized_items:
        print(item)

# Analizar nuevo gdf
for item in list(joined_types_cc.itemname.unique()):
    if item not in analized_items:
        print(item)

In [25]:
joined_types_ccc = joined_types_cc.copy()

for material in material_itemname_dicc.keys():
    for item in material_itemname_dicc[material]:
        idx = joined_types_ccc.itemname == item
        joined_types_ccc.loc[idx,'material'] = material
        
# Show
print(joined_types_ccc.shape)
joined_types_ccc.head(1)

(24059, 9)


,id,itemname,tipo,quantity,latitude,longitude,altitude,datetime,material
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31,plastico


In [26]:
# Order columns
joined_types_ccc = joined_types_ccc[['id','itemname','tipo','material','quantity','latitude','longitude','altitude','datetime']]

## Download and data treatment for missing csv data points (Found in '30_05.csv')

### Download missing csv data ('30_05.csv')

In [27]:
tmp_dir = f'../../data/external/temporal_todocker/arroyo_vivo/input_csv/'
doc_30_05 = pd.read_csv(tmp_dir+'30_05.csv')

# Show
print(doc_30_05.columns)
doc_30_05.head(1)

Index(['itemname', 'material', 'id', 'quantity', 'description', 'latitude',
       'longitude', 'altitude'],
      dtype='object')


,itemname,material,id,quantity,description,latitude,longitude,altitude
0,Other Metal,METAL,Metal,1,NaN,25.641804,-100.314824,562.835693


### Format missing csv data

In [29]:
# Create index with unique numbers column
missing_doc = doc_30_05.reset_index()

# Rename columns
missing_doc.rename(columns={'material':'tipo',
                          'id':'material',
                           'index':'id'},inplace=True)

# Make data in 'material' column x.lower()
missing_doc['material'] = missing_doc['material'].apply(lambda x: x.lower())

# Add datetime column
missing_doc['datetime'] = pd.to_datetime('20230530', format='%Y%m%d')

# Delete description column
missing_doc.drop(columns=['description'],inplace=True)

# Show
print(missing_doc.shape)
missing_doc.head(1)

(315, 9)


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime
0,0,Other Metal,METAL,metal,1,25.641804,-100.314824,562.835693,2023-05-30


### Fix column itemname (Dictionary with all previously found variations)

In [30]:
missing_doc_2 = missing_doc.copy()

fix_names = {'Latas de aluminio o estaño':['Aluminum or tin cans'],
             'Papel de aluminio':['papel de aluminio'],
             'Baterías':['baterias'],
             'Cartón acanalado':['carton acanalado'],
             'Cartón recubierto':['carton recubierto'],
             'Envoltura de alimentos de papel sin recubrimiento':['Food wrappers (paper)'],
             'Fragmentos de vidrio o cerámica':['Fragmentos de vidrio o cerÃ¡mica','Glass fragments'],
             'Fragmentos de electrónicos':['Fragmentos de electronicos'],
             'Otros desechos electrónicos':['Otros desechos electronicos'],
             'Materiales de construcción':['Materiales de cconstruccion','Materiales de Construccion','Lumber/Building Materials'],
             'Otros artículos de construcción':['Otros artÃ­culos de cconstruccion','Otros artículos de Construccion','Otros articulos de Construccion'],
             'Palito de paleta':['Pallets'],
             'Fragmentos de metal':['Metal fragments'],
             'Otro metal':['Other Metal'],
             'Tapas o lengüetas de metal para botellas':['Tapas o lenguetas de metal para botellas'],
             'Otros desechos orgánicos':['Otros desechos organico'],
             'Bolsas de papel':['Paper Bags'],
             'Papel':['papel','Paper and Cardboard'],
             'Bolsa de plástico (otra)':['Bolsa de plastico (otra)','Bolsa de Plastico (otra)'],
             'Bolsa de plástico para comestibles':['Bolsa de plastico para comestibles','Bolsa de Plastico para comestibles'],
             'Botella de plástico':['Botella de plastico','Botella de Plastico'],
             'Cuerda de plástico':['Cuerda de plastico'],
             'Cuerda de plástico/ cinta o correas de embalaje':['Cuerda de plastico/ cinta o correas de embalaje','Cuerda de Plastico/ cinta o correas de embalaje'],
             'Envoltorio de plástico para alimentos':['Envoltorio de plastico para alimentos','Envoltorio de Plastico para alimentos'],
             'Fragmentos de película de plástico':['Fragmentos de peliculade plastico','Fragmentos de película de Plastico'],
             'Fragmentos de plástico duro':['Fragmentos de plastico duro','Fragmentos de Plastico duro'],
             'Otro plástico':['Otro plastico','Otro Plastico'],
             'Otro plástico relacionado con los alimentos':['Otro plastico relacionado con los alimentos','Otro Plastico relacionado con los alimentos'],
             'Red de plástico o piezas de red':['Red de plastico o piezas de red','Tapa de botella de Plastico'],
             'Tapa de botella de plástico':['Tapa de botella de plastico'],
             'Utensilios de plástico':['Utensilios de plastico','Utensilios de Plastico'],
             'Champú u otro recipiente de HDPE':['ChampÃº u otro recipiente de HDPE'],
             'Cotonetes de algodón':['Cotonetes de algodÃ³n'],
             'Máscaras faciales':['MÃ¡scaras faciales','Fabric Face Masks'],
             'Otra tela':['Other Cloth'],
             'Piezas de tela':['Fabric pieces'],
             'Tazón de comida para llevar':['TazÃ³n de comida para llevar'],
             'Vasos o tapas de unicel o plástico':['Vasos o tapas de unicel o plastico','Vasos o tapas de unicel o Plastico'],
             'Botella de vidrio':['Glass Jars'],
             'Otro papel':['Other Paper & Lumber'],
             'Bolsas de papel':['Paper Bags'],
             'Toallas o trapos':['Towels or rags'],
             'Ropa o zapatos':['Clothing and Shoes'],
             'Tienda de café / té':['Tienda de cafÃ© / tÃ©']
            }

for k in fix_names.keys():
    for item in fix_names[k]:
        idx = missing_doc_2.itemname == item
        missing_doc_2.loc[idx,'itemname'] = k
        
# Show
print(missing_doc_2.shape)
missing_doc_2.head(1)

(315, 9)


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime
0,0,Otro metal,METAL,metal,1,25.641804,-100.314824,562.835693,2023-05-30


### Fix column tipo

In [32]:
missing_doc_3 = missing_doc_2.copy()

# For each item
for item in list(missing_doc_3.itemname.unique()):

    # Find its assigned 'tipo' in original mdt csvs
    tmp_gdf = joined_types_ccc.loc[joined_types_ccc.itemname == item]
    tipo_list = tmp_gdf.tipo.unique()
    if len(tipo_list) == 1:
        tipo = tipo_list[0]
        idx = (missing_doc_3.itemname == item)
        missing_doc_3.loc[idx,'tipo'] = tipo
    
    elif len(tipo_list) == 0:
        if item == 'Bolsas de papel':
            tipo = 'Papel'
        elif item == 'Materiales de Construcción':
            tipo = 'Materiales de Construcción'
        elif item == 'Plastico para cerveza':
            tipo = 'Productos Plasticos para Comida'
        elif item == 'Fragmentos de Plástico':
            tipo = 'Fragmentos de Plastico  '
        elif item == 'Toallas o trapos':
            tipo = 'Tela'
        elif item == 'Ropa o zapatos':
            tipo = 'Tela'
        elif item == 'Tela':
            tipo = 'Tela'
        else:
            print('Problema con:')
            print(item)
            print(tipo)
        
        idx = (missing_doc_3.itemname == item)
        missing_doc_3.loc[idx,'tipo'] = tipo        
        
# Show
print(missing_doc_3.shape)
missing_doc_3.head(1)

(315, 9)


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime
0,0,Otro metal,Metales,metal,1,25.641804,-100.314824,562.835693,2023-05-30


In [33]:
# Review result
missing_doc_3.tipo.unique()

array(['Metales', 'Papel', 'Otro', 'Materiales de Construcción',
       'Fragmentos de Plástico  ', 'Productos Plasticos para Comida',
       'Productos Plásticos para Comida', 'EPI', 'Tela', 'Vidrio'],
      dtype=object)

## Data treatment for all data

Join data

In [34]:
joined_types_cccc = pd.concat([joined_types_ccc,missing_doc_3])

# Show
print(joined_types_cccc.shape)
joined_types_cccc.head(1)

(24374, 9)


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,plastico,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


### Data treatment - Format and delete duplicates

In [35]:
# Delete duplicates
garbage = joined_types_cccc.drop_duplicates() #subset=['id','itemname','tipo','material','quantity','latitude','longitude','datetime']

# Show
print(garbage.shape)
garbage.head(1)

(14497, 9)


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,plastico,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31


## Format

In [36]:
# Add 'jornada' column
garbage['jornada'] = garbage['datetime'].dt.strftime('%y%m%d')

# Show
print(garbage.shape)
garbage.head(1)

(14497, 10)


/tmp/ipykernel_1000/2155304246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  garbage['jornada'] = garbage['datetime'].dt.strftime('%y%m%d')


,id,itemname,tipo,material,quantity,latitude,longitude,altitude,datetime,jornada
0,2251010,Bolsa de plástico para comestibles,Productos Plásticos para Comida,plastico,5,25.641495,-100.293776,530.400024,2022-10-29 14:05:31,221029


## Save to local for inspection

In [38]:
garbage.to_csv("../../data/external/temporal_fromjupyter/arroyo_vivo/residuos_arroyovivo_jornada.csv",index=False)

## Load garbage (clipped and inspected geopackage in GIS) and upload to db

In [39]:
garbage_dir = f'../../data/external/temporal_todocker/arroyo_vivo/arroyovivo_residuosjornada_cuenca_point_32614.gpkg'
garbage_inspected = gpd.read_file(garbage_dir)
garbage_inspected = garbage_inspected.to_crs("EPSG:4326")

aup.gdf_to_db_slow(garbage_inspected, 'arroyovivo_residuosjornada_point','projects_research', if_exists='replace')